In [ ]:
# Copyright 2021 Google LLC
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     https://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.

# Vertex AI迁移：AutoML视频分类

<table align="left">

  <td>
    <a href="https://colab.research.google.com/github/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-automl-video-classification-batch-prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/colab-logo-32px.png" alt="Colab logo"> 在Colab中运行
    </a>
  </td>
  <td>
    <a href="https://github.com/GoogleCloudPlatform/vertex-ai-samples/blob/main/notebooks/official/migration/sdk-automl-video-classification-batch-prediction.ipynb">
      <img src="https://cloud.google.com/ml-engine/images/github-logo-32px.png" alt="GitHub logo">
      在GitHub上查看
    </a>
  </td>
  <td>
<a href="https://console.cloud.google.com/vertex-ai/workbench/deploy-notebook?download_url=https://raw.githubusercontent.com/GoogleCloudPlatform/vertex-ai-samples/main/notebooks/official/migration/sdk-automl-video-classification-batch-prediction.ipynb" target='_blank'>
      <img src="https://lh3.googleusercontent.com/UiNooY4LUgW_oTvpsNhPpQzsstV5W8F7rYgxgGBD85cWJoLmrOzhVs_ksK_vgx40SHs7jCqkTkCk=e14-rj-sc0xffffff-h130-w32" alt="Vertex AI logo">
      在Vertex AI Workbench中打开
    </a>
  </td>                                                                                               
</table>
<br/><br/><br/>

## 概述

本教程演示了如何使用Python的Vertex AI SDK训练AutoML视频分类模型并进行批量预测。

了解更多关于[迁移到Vertex AI](https://cloud.google.com/vertex-ai/docs/start/migrating-to-vertex-ai)和[视频数据分类](https://cloud.google.com/vertex-ai/docs/training-overview#classification_for_videos)。

### 目标

在本教程中，您将学习如何使用`AutoML`训练视频模型，并使用`Vertex AI Batch Prediction`来进行批量预测。

此教程使用以下Google Cloud ML服务：

- `AutoML`
- `Vertex AI Batch Prediction`
- `Vertex AI Model` 资源
- `Vertex AI Endpoint` 资源

执行的步骤包括：

- 训练一个AutoML视频分类模型。
- 进行批量预测。

数据集

本教程使用的数据集是来自[麻省理工学院](http://cbcl.mit.edu/publications/ps/Kuehne_etal_iccv11.pdf)的[人体运动数据集](https://TODO)。您在本教程中将使用的数据集版本存储在公共云存储桶中。

### 成本

本教程使用 Google Cloud 的计费组件：

* Vertex AI
* Cloud Storage

了解 [Vertex AI 的定价](https://cloud.google.com/vertex-ai/pricing) 和 [Cloud Storage 的定价](https://cloud.google.com/storage/pricing)，并使用 [定价计算器](https://cloud.google.com/products/calculator/) 根据您的预期使用量生成成本估算。

安装

安装执行此笔记本所需的软件包。

In [ ]:
import os

! pip3 install --upgrade --quiet google-cloud-aiplatform 

if os.getenv("IS_TESTING"):
    ! pip3 install --upgrade --quiet google-cloud-storage tensorflow

### 仅合作：取消下面的单元格注释以重新启动内核

In [ ]:
# Automatically restart kernel after installs so that your environment can access the new packages
# import IPython

# app = IPython.Application.instance()
# app.kernel.do_shutdown(True)

在您开始之前

### 设置项目 ID

**如果您不知道您的项目 ID**，请尝试以下操作：
* 运行 `gcloud config list`。
* 运行 `gcloud projects list`。
* 查看支持页面：[查找项目 ID](https://support.google.com/googleapi/answer/7014113)。

In [ ]:
PROJECT_ID = "[your-project-id]"  # @param {type:"string"}

# Set the project id
! gcloud config set project {PROJECT_ID}

区域

您还可以更改 Vertex AI 使用的 `REGION` 变量。了解有关 [Vertex AI 区域](https://cloud.google.com/vertex-ai/docs/general/locations) 的更多信息。

In [ ]:
REGION = "us-central1"  # @param {type: "string"}

UUID

如果您在一个实时教程会话中，您可能正在使用共享的测试账户或项目。为了避免用户在创建的资源中发生名称冲突，您为每个实例会话创建一个uuid，并将其附加到您在本教程中创建的资源的名称上。

In [ ]:
import random
import string


# Generate a uuid of a specifed length(default=8)
def generate_uuid(length: int = 8) -> str:
    return "".join(random.choices(string.ascii_lowercase + string.digits, k=length))


UUID = generate_uuid()

### 验证您的谷歌云帐号

根据您的Jupyter环境，您可能需要手动进行验证。请按照以下相关说明操作。

**1. Vertex AI Workbench**
* 不需要进行任何操作，因为您已经过验证。

**2. 本地JupyterLab实例，请取消注释并运行:**

In [ ]:
# ! gcloud auth login

3. 协作、取消注释并运行：

In [ ]:
# from google.colab import auth
# auth.authenticate_user()

查看如何在https://cloud.google.com/storage/docs/gsutil/commands/iam#ch-examples 给您的服务账号授予云存储权限。

创建一个云存储桶

创建一个存储桶来存储中间产物，如数据集。

In [ ]:
BUCKET_URI = f"gs://your-bucket-name-{PROJECT_ID}-unique"  # @param {type:"string"}

只有当您的存储桶尚不存在时：运行以下单元格以创建您的云存储存储桶。

In [ ]:
! gsutil mb -l {REGION} {BUCKET_URI}

### 设置变量

接下来，设置一些在本教程中使用的变量。

### 导入库并定义常量

In [ ]:
import os
import sys

import google.cloud.aiplatform as aip

## 初始化 Python 的 Vertex SDK

为您的项目和对应的存储桶初始化 Python 的 Vertex SDK。

In [ ]:
aip.init(project=PROJECT_ID, staging_bucket=BUCKET_URI)

云存储训练数据的位置。

现在将变量`IMPORT_FILE`设置为云存储中CSV索引文件的位置。

In [ ]:
IMPORT_FILE = "gs://automl-video-demo-data/hmdb_split1_train_40_mp4_gs.csv"

快速查看你的数据

本教程使用存储在公共云存储桶中的MIT人体运动数据集的版本，使用CSV索引文件。

首先快速查看数据。通过统计CSV索引文件中的行数（`wc -l`）来计算示例的数量，然后查看前几行数据。

In [ ]:
if "IMPORT_FILES" in globals():
    FILE = IMPORT_FILES[0]
else:
    FILE = IMPORT_FILE

count = ! gsutil cat $FILE | wc -l
print("Number of Examples", int(count[0]))

print("First 10 rows")
! gsutil cat $FILE | head

In [ ]:
import datetime

start = datetime.datetime.now()

创建一个数据集

### [datasets.create-dataset-api](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api)

### [数据集创建API](https://cloud.google.com/vertex-ai/docs/datasets/create-dataset-api)

### 创建数据集

接下来，使用`VideoDataset`类的`create`方法创建`Dataset`资源，该方法接受以下参数：

- `display_name`：`Dataset`资源的人类可读名称。
- `gcs_source`：一个或多个数据集索引文件的列表，用于将数据项导入`Dataset`资源中。

此操作可能需要几分钟的时间。

In [ ]:
dataset = aip.VideoDataset.create(
    display_name="MIT Human Motion" + "_" + UUID,
    gcs_source=[IMPORT_FILE],
    import_schema_uri=aip.schema.dataset.ioformat.video.classification,
)

print(dataset.resource_name)

*示例输出：*

    信息：google.cloud.aiplatform.datasets.dataset：创建VideoDataset
    信息：google.cloud.aiplatform.datasets.dataset：创建VideoDataset支持LRO：projects/759209241365/locations/us-central1/datasets/5948525032035581952/operations/6913187331100901376
    信息：google.cloud.aiplatform.datasets.dataset：VideoDataset已创建。资源名称：projects/759209241365/locations/us-central1/datasets/5948525032035581952
    信息：google.cloud.aiplatform.datasets.dataset：在另一个会话中使用此VideoDataset：
    信息：google.cloud.aiplatform.datasets.dataset：ds = aiplatform.VideoDataset('projects/759209241365/locations/us-central1/datasets/5948525032035581952')
    信息：google.cloud.aiplatform.datasets.dataset：导入VideoDataset数据：projects/759209241365/locations/us-central1/datasets/5948525032035581952
    信息：google.cloud.aiplatform.datasets.dataset：导入VideoDataset数据支持LRO：projects/759209241365/locations/us-central1/datasets/5948525032035581952/operations/6800597340416638976

训练一个模型

### [training.automl-api](https://cloud.google.com/vertex-ai/docs/training/automl-api)

### [训练.automl-api](https://cloud.google.com/vertex-ai/docs/training/automl-api)

### 创建和运行训练流水线

要训练一个自动机器学习模型，您需要执行两个步骤：1）创建一个训练流水线，2）运行该流水线。

#### 创建训练流水线

使用`AutoMLVideoTrainingJob`类创建一个自动机器学习训练流水线，指定以下参数：

- `display_name`：`TrainingJob`资源的可读名称。
- `prediction_type`：要为模型训练的类型任务。
  - `classification`：视频分类模型。
  - `object_tracking`：视频目标跟踪模型。
  - `action_recognition`：视频动作识别模型。

实例化的对象是训练流水线的有向无环图(DAG)。

In [ ]:
dag = aip.AutoMLVideoTrainingJob(
    display_name="hmdb_" + UUID,
    prediction_type="classification",
)

print(dag)

*示例输出：*

<google.cloud.aiplatform.training_jobs.AutoMLVideoTrainingJob object at 0x7fc3b6c90f10> 

（AI 平台训练作业自动视频培训作业对象 at 0x7fc3b6c90f10）

#### 运行训练流程

接下来，您可以运行DAG以通过调用`run`方法启动训练作业，参数如下：

- `dataset`：要训练模型的`数据集`资源。
- `model_display_name`：训练模型的人类可读名称。
- `training_fraction_split`：用于训练的数据集百分比。
- `test_fraction_split`：用于测试（留出数据）的数据集百分比。

`run`方法完成后会返回`Model`资源。

训练流程的执行可能需要超过24小时。

In [ ]:
if os.getenv("IS_TESTING"):
    sys.exit(0)

In [ ]:
model = dag.run(
    dataset=dataset,
    model_display_name="hmdb_" + UUID,
    training_fraction_split=0.8,
    test_fraction_split=0.2,
)

*示例输出：*

    信息：google.cloud.aiplatform.training_jobs:查看训练：
    https://console.cloud.google.com/ai/platform/locations/us-central1/training/6090621516762841088?project=759209241365
    信息：google.cloud.aiplatform.training_jobs:AutoMLVideoTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/6090621516762841088 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs:AutoMLVideoTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/6090621516762841088 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs:AutoMLVideoTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/6090621516762841088 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs:AutoMLVideoTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/6090621516762841088 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs:AutoMLVideoTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/6090621516762841088 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs:AutoMLVideoTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/6090621516762841088 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs:AutoMLVideoTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/6090621516762841088 当前状态：
    PipelineState.PIPELINE_STATE_RUNNING
    信息：google.cloud.aiplatform.training_jobs:AutoMLVideoTrainingJob projects/759209241365/locations/us-central1/trainingPipelines/6090621516762841088 当前状态：
    ...
    信息：google.cloud.aiplatform.training_jobs:AutoMLVideoTrainingJob 运行完成。资源名称：projects/759209241365/locations/us-central1/trainingPipelines/6090621516762841088
    信息：google.cloud.aiplatform.training_jobs:模型可在 projects/759209241365/locations/us-central1/models/1899701006099283968 中找到。

In [ ]:
end = datetime.datetime.now()
end - start

评估模型

### [项目位置模型评估列表](https://cloud.devsite.corp.google.com/ai-platform-unified/docs/reference/rest/v1beta1/projects.locations.models.evaluations/list)

## 回顾模型评估分数
在您的模型训练完成后，您可以查看其评估分数。

首先，您需要获取对新模型的引用。与数据集一样，您可以使用在部署模型时创建的模型变量的引用，或者您也可以列出项目中的所有模型。

In [ ]:
# Get model resource ID
models = aip.Model.list(filter="display_name=hmdb_" + UUID)

# Get a reference to the Model Service client
client_options = {"api_endpoint": f"{REGION}-aiplatform.googleapis.com"}
model_service_client = aip.gapic.ModelServiceClient(client_options=client_options)

model_evaluations = model_service_client.list_model_evaluations(
    parent=models[0].resource_name
)
model_evaluation = list(model_evaluations)[0]
print(model_evaluation)

*示例输出:*

    名称: "projects/759209241365/locations/us-central1/models/623915674158235648/evaluations/4280507618583117824"
    指标模式 URI: "gs://google-cloud-aiplatform/schema/modelevaluation/classification_metrics_1.0.0.yaml"
    指标 {
      struct_value {
        fields {
          键: "auPrc"
          值 {
            number_value: 0.9891107
          }
        }
        fields {
          键: "confidenceMetrics"
          值 {
            list_value {
              values {
                struct_value {
                  fields {
                    键: "precision"
                    值 {
                      number_value: 0.2
                    }
                  }
                  fields {
                    键: "recall"
                    值 {
                      number_value: 1.0
                    }
                  }
                }
              }

进行批量预测

### [predictions.batch-prediction](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

### [predictions.batch-prediction](https://cloud.google.com/vertex-ai/docs/predictions/batch-predictions)

获取测试项

现在对您的Vertex模型执行批量预测。 您将使用数据集中的任意示例作为测试项目。 不必担心这些示例可能已经用于训练模型-我们只是想演示如何进行预测。

In [ ]:
test_items = ! gsutil cat $IMPORT_FILE | head -n2

if len(test_items[0]) == 5:
    _, test_item_1, test_label_1, _, _ = str(test_items[0]).split(",")
    _, test_item_2, test_label_2, _, _ = str(test_items[1]).split(",")
else:
    test_item_1, test_label_1, _, _ = str(test_items[0]).split(",")
    test_item_2, test_label_2, _, _ = str(test_items[1]).split(",")


print(test_item_1, test_label_1)
print(test_item_2, test_label_2)

### 创建一个批量输入文件

现在创建一个批量输入文件，将其存储在您的本地云存储桶中。批量输入文件可以是CSV格式或JSONL格式。在本教程中，您将使用JSONL格式。对于JSONL文件，您需要为每个视频的每一行创建一个字典条目。该字典包含以下键值对：

- `content`: 视频的云存储路径。
- `mimeType`: 内容类型。在我们的例子中，是一个`avi`文件。
- `timeSegmentStart`: 在视频中进行预测的起始时间戳。*注意*，时间戳必须以字符串形式指定，并以s（秒）、m（分钟）或h（小时）结尾。
- `timeSegmentEnd`: 在视频中进行预测的结束时间戳。

In [ ]:
import json

import tensorflow as tf

gcs_input_uri = BUCKET_URI + "/test.jsonl"
with tf.io.gfile.GFile(gcs_input_uri, "w") as f:
    data = {
        "content": test_item_1,
        "mimeType": "video/avi",
        "timeSegmentStart": "0.0s",
        "timeSegmentEnd": "5.0s",
    }
    f.write(json.dumps(data) + "\n")
    data = {
        "content": test_item_2,
        "mimeType": "video/avi",
        "timeSegmentStart": "0.0s",
        "timeSegmentEnd": "5.0s",
    }
    f.write(json.dumps(data) + "\n")

print(gcs_input_uri)
! gsutil cat $gcs_input_uri

### 进行批量预测请求

现在您的模型资源已经训练完毕，您可以通过调用batch_predict()方法进行批量预测，使用以下参数：

- `job_display_name`：批量预测作业的可读名称。
- `gcs_source`：一个或多个批量请求输入文件的列表。
- `gcs_destination_prefix`：用于存储批量预测结果的Cloud Storage位置。
- `sync`：如果设置为True，调用将会在等待异步批量作业完成时阻塞。

In [ ]:
batch_predict_job = model.batch_predict(
    job_display_name="hmdb_" + UUID,
    gcs_source=gcs_input_uri,
    gcs_destination_prefix=BUCKET_URI,
    sync=False,
)

print(batch_predict_job)

*示例输出：*

    INFO:google.cloud.aiplatform.jobs:创建BatchPredictionJob
    <google.cloud.aiplatform.jobs.BatchPredictionJob object at 0x7f806a6112d0>正在等待上游依赖项完成。
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob已创建。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296
    INFO:google.cloud.aiplatform.jobs:要在另一个会话中使用此BatchPredictionJob：
    INFO:google.cloud.aiplatform.jobs:bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296')
    INFO:google.cloud.aiplatform.jobs:查看Batch Prediction Job：
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/5110965452507447296?project=759209241365
    INFO:google.cloud.aiplatform.jobs:BatchPredictionJob项目/759209241365/locations/us-central1/batchPredictionJobs/5110965452507447296当前状态：
    JobState.JOB_STATE_RUNNING

### 等待批处理预测任务完成

接下来，等待批处理任务完成。或者，可以在`batch_predict()`方法中设置参数`sync`为`True`，以阻塞直到批处理预测任务完成。

In [ ]:
batch_predict_job.wait()

*示例输出：*

    信息：google.cloud.aiplatform.jobs：已创建BatchPredictionJob。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328
    信息：google.cloud.aiplatform.jobs：要在另一个会话中使用此BatchPredictionJob：
    信息：google.cloud.aiplatform.jobs：bpj = aiplatform.BatchPredictionJob('projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328')
    信息：google.cloud.aiplatform.jobs：查看批量预测作业：
    https://console.cloud.google.com/ai/platform/locations/us-central1/batch-predictions/181835033978339328?project=759209241365
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_RUNNING
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328 当前状态：
    JobState.JOB_STATE_SUCCEEDED
    信息：google.cloud.aiplatform.jobs：BatchPredictionJob 运行完成。资源名称：projects/759209241365/locations/us-central1/batchPredictionJobs/181835033978339328

获取预测结果

接下来，从已完成的批量预测作业中获取结果。

结果会被写入您在批量预测请求中指定的云存储输出存储桶中。您可以调用iter_outputs()方法获取生成结果的每个云存储文件的列表。每个文件以JSON格式包含一个或多个预测请求：

- `content`：预测请求。
- `prediction`：预测响应。
 - `ids`：每个预测请求的内部分配的唯一标识符。
 - `displayNames`：每个类别标签的类名。
 - `confidences`：每个类别标签的预测置信度，介于0和1之间。
 - `timeSegmentStart`：视频中开始视频序列的时间偏移。
 - `timeSegmentEnd`：视频中结束视频序列的时间偏移。

In [ ]:
import json

import tensorflow as tf

bp_iter_outputs = batch_predict_job.iter_outputs()

prediction_results = list()
for blob in bp_iter_outputs:
    if blob.name.split("/")[-1].startswith("prediction"):
        prediction_results.append(blob.name)

tags = list()
for prediction_result in prediction_results:
    gfile_name = f"gs://{bp_iter_outputs.bucket.name}/{prediction_result}"
    with tf.io.gfile.GFile(name=gfile_name, mode="r") as gfile:
        for line in gfile.readlines():
            line = json.loads(line)
            print(line)
            break

{'instance': {'content': 'gs://automl-video-demo-data/hmdb51/Acrobacias_de_un_fenomeno_cartwheel_f_cm_np1_ba_bad_8.avi', 'mimeType': 'video/avi', 'timeSegmentStart': '0.0s', 'timeSegmentEnd': '5.0s'}, 'prediction': [{'id': '4517318233950257152', 'displayName': 'cartwheel', 'type': 'segment-classification', 'timeSegmentStart': '0s', 'timeSegmentEnd': '5s', 'confidence': 0.7450977}, {'id': '6823161243163951104', 'displayName': 'pullup', 'type': 'segment-classification', 'timeSegmentStart': '0s', 'timeSegmentEnd': '5s', 'confidence': 0.07339612}, {'id': '2211475224736563200', 'displayName': 'golf', 'type': 'segment-classification', 'timeSegmentStart': '0s', 'timeSegmentEnd': '5s', 'confidence': 0.065019816}, {'id': '9129004252377645056', 'displayName': 'kick_ball', 'type': 'segment-classification', 'timeSegmentStart': '0s', 'timeSegmentEnd': '5s', 'confidence': 0.06463309}, {'id': '121804997636653056', 'displayName': 'ride_horse', 'type': 'segment-classification', 'timeSegmentStart': '0s', 'timeSegmentEnd': '5s', 'confidence': 0.05185325}]}
{'instance': {'content': 'gs://automl-video-demo-data/hmdb51/_Rad_Schlag_die_Bank__cartwheel_f_cm_np1_le_med_0.avi', 'mimeType': 'video/avi', 'timeSegmentStart': '0.0s', 'timeSegmentEnd': '5.0s'}, 'prediction': [{'id': '4517318233950257152', 'displayName': 'cartwheel', 'type': 'segment-classification', 'timeSegmentStart': '0s', 'timeSegmentEnd': '5s', 'confidence': 0.76310456}, {'id': '2211475224736563200', 'displayName': 'golf', 'type': 'segment-classification', 'timeSegmentStart': '0s', 'timeSegmentEnd': '5s', 'confidence': 0.06767218}, {'id': '6823161243163951104', 'displayName': 'pullup', 'type': 'segment-classification', 'timeSegmentStart': '0s', 'timeSegmentEnd': '5s', 'confidence': 0.05853845}, {'id': '9129004252377645056', 'displayName': 'kick_ball', 'type': 'segment-classification', 'timeSegmentStart': '0s', 'timeSegmentEnd': '5s', 'confidence': 0.055601567}, {'id': '121804997636653056', 'displayName': 'ride_horse', 'type': 'segment-classification', 'timeSegmentStart': '0s', 'timeSegmentEnd': '5s', 'confidence': 0.055083193}]}

清理

要清理本项目中使用的所有 Google Cloud 资源，可以删除用于本教程的[Google Cloud 项目](https://cloud.google.com/resource-manager/docs/creating-managing-projects#shutting_down_projects)。

否则，您可以删除在本教程中创建的各个资源。

In [ ]:
delete_bucket = False

# Delete the dataset using the Vertex dataset object

dataset.delete()

# Delete the model using the Vertex model object
model.delete()

# Delete the AutoML or Pipeline trainig job

dag.delete()

# Delete the batch prediction job using the Vertex batch prediction object
batch_predict_job.delete()

if delete_bucket or os.getenv("IS_TESTING"):
    ! gsutil rm -r $BUCKET_URI